In [4]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("srikanthsri/medical_biological", split="train")


Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [5]:
print(dataset[0])  # Print the first example in the dataset


{'text': '###Question : What is (are) Hepatitis B: What Asian and Pacific Islander Americans Need to Know ? ###Answer : Hepatitis B is a liver disease spread through contact with blood, semen, or other body fluids from a person infected with the hepatitis B virus. The disease is most commonly spread from an infected mother to her infant at birth. Hepatitis B is also spread through sex, wound-to-wound contact, and contact with items that may have blood on them, such as shaving razors, toothbrushes, syringes, and tattoo and body piercing needles.\n                \nHepatitis B is not spread through casual contact such as shaking hands or hugging; nor is it spread by sharing food or beverages, by sneezing and coughing, or through breastfeeding.'}


In [6]:
dataset.to_csv("medical_biological.csv")  # Save as CSV

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

3885681

In [16]:
import pandas as pd
import re

# Load the CSV file
df = pd.read_csv('medical_biological.csv')

# Function to clean and split the text
def split_question_answer(text):
    # Remove the starting ### and clean up any unnecessary whitespace
    text = text.replace("###", "").strip()
    
    # Attempt to split based on either '###Answer : ' or 'Answer : ' as the separator
    if "Answer :" in text:
        # Split using 'Answer :' as a flexible separator
        question_part, answer_part = re.split(r"Answer :", text, maxsplit=1)
        
        # Remove specific extraneous text from the question part if it exists
        question_part = re.sub(r"B: What Asian and Pacific Islander Americans Need to Know ?", "", question_part)
        
        # Clean up any remaining labels and extra spaces
        question = question_part.replace("Question :", "").strip()
        answer = answer_part.strip()
        
        # Return None if either question or answer is empty after processing
        if not question or not answer:
            print("Empty question or answer in row:", text)  # Debugging info
            return None, None
        
        return question, answer
    else:
        print("Separator not found in row:", text)  # Debugging info
        return None, None

# Apply the function to each row in the DataFrame
df[['Question', 'Answer']] = df['text'].apply(lambda x: pd.Series(split_question_answer(x)))

# Drop rows where question or answer is missing
df = df.dropna(subset=['Question', 'Answer'])

# Drop the original text column
df = df.drop(columns=['text'])

# Save to a new CSV file
df.to_csv('formatted_medical_biological.csv', index=False)
print("File saved as 'formatted_medical_biological.csv'")

File saved as 'formatted_medical_biological.csv'
